<a href="https://colab.research.google.com/github/Chicane2002/Projects/blob/main/Pet_Classifier_resnet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#The following changes have been made in the given code:
#1. Batch normalisation
#2. Hyperprameter tuning
#3. Learning rate scheduler
#4. Manual grid parameter tuning of epochs

#Accuracy increased from 83% to 88.4%

import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

images_fp = '/content/drive/MyDrive/images'
dataset_name = 'tanlikesmath/the-oxfordiiit-pet-dataset'

image_names = [os.path.basename(file) for file in glob.glob(os.path.join(images_fp, '*.jpg'))]
#image_names

In [ ]:
labels = set([' '.join(name.split('_')[:-1:]) for name in image_names])
i=0
l={}
while i<len(labels):
  l[list(labels)[i]]=i
  i+=1
def label_encode(label):
  return l[label]


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
features = []
labels = []
image_size=(224,224)

for name in image_names:
  label=' '.join(name.split('_')[:-1:])
  label_encoded=label_encode(label)
  if label_encoded != None:
    img = load_img(os.path.join(images_fp,name))
    img = tf.image.resize_with_pad(img_to_array(img,dtype='uint8'),*image_size).numpy().astype('uint8')
    image=np.array(img)
    features.append(image)
    labels.append(label_encoded)



In [ ]:
features_array=np.array(features)
labels_array=np.array(labels)
labels_one_hot=pd.get_dummies(labels_array)

In [ ]:
from sklearn.model_selection import train_test_split
# Train:60%,val:25%,test:20%
X_train, X_test, y_train, y_test = train_test_split(features_array, labels_one_hot, test_size=0.2, random_state=48)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=48)

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input as pp_i
from keras.layers import RandomFlip, RandomRotation, BatchNormalization
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.callbacks import LearningRateScheduler

data_augmentation = keras.Sequential(
  [
    RandomFlip("horizontal_and_vertical"),
    RandomRotation(0.24),
  ]
)

preprocess_input = pp_i

prediction_layers= Dense(37,activation='softmax')

resnet_model = ResNet50(include_top=False,pooling='avg',weights='imagenet')
resnet_model.trainable=False

# Building model

inputs = keras.Input(shape=(224, 224, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = resnet_model(x, training=False)
x = Dropout(0.2)(x)
x = BatchNormalization()(x)  # Mini-batch normalization
outputs = prediction_layers(x)
model = keras.Model(inputs, outputs)

model.compile(optimizer=Adam(learning_rate=0.005),
              loss=CategoricalCrossentropy(),
              metrics=['accuracy'])
def lr_schedule(epoch):
    initial_lr = 0.005
    if epoch > 10:
        return initial_lr * 0.1
    return initial_lr
# Hyper parameter tuning for learning rate
lr_scheduler = LearningRateScheduler(lr_schedule)

# Model training with hyperparameter tuning
for e in range(10,31):
  model_history = model.fit(X_train, y_train,
                            epochs=e,
                            batch_size=32,
                            validation_data=(X_val, y_val),
                            callbacks=[lr_scheduler])